In [30]:
import sys
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import re

In [31]:
dataset = 'NJH2015'

print('Loading raw data for', dataset, '...')
data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/1-s2.0-S1550413115004581-mmc2.csv', header=0)
print('Raw data loaded.')
data

Loading raw data for NJH2015 ...
Raw data loaded.


,Sequence_Localization Probability,Proteins,GeneNames,Phosphorylation position (>90% localization),Protein Descriptions,Protein Group Accessions,Modifications,iTRAQ Subject 1 115/114 (Exercise/Basal) (normalized),iTRAQ Subject 2 117/116 (Exercise/Basal) (normalized),iTRAQ Subject 3 116/117 (Exercise/Basal) (normalized),...,NetworKIN Score(s),Sequence Window,PhosphoSitePlus Match,PhosphoSitePlus Group ID,Experimental Upstream Kinase,In Domain,On Function,On Process,On Protein Interaction,On Other Interaction
0,rAsmLEDSGSk_S(3): >90%; S(8): <10%; S(10): <10%,A0AVK6,E2F8,A0AVK6_S608,Transcription factor E2F8 OS=Homo sapiens GN=E...,A0AVK6,N-Term(TMT6plex); S3(Phospho); M4(Oxidation); ...,na,na,na,...,na;na;na;na;na;na;na;na;na;na;na;na,ERGSKRASMLEDSGS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,rAsMLEDsGSk_S(3): >90%; S(8): >90%; S(10): <10%,A0AVK6,E2F8,A0AVK6_S608;A0AVK6_S613,Transcription factor E2F8 OS=Homo sapiens GN=E...,A0AVK6,N-Term(TMT6plex); S3(Phospho); S8(Phospho); K1...,na,na,na,...,na;na;na;na;na;na;na;na;na;na;na;na,ERGSKRASMLEDSGS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rPsVSk_S(3): >90%; S(5): <10%,A0FGR8;A0FGR8-2;A0FGR8-4;A0FGR8-5;A0FGR8-6;F2Z...,ESYT2,A0FGR8_S676;A0FGR8-2_S648;A0FGR8-4_S472;A0FGR8...,Extended synaptotagmin-2 OS=Homo sapiens GN=ES...,A0FGR8;A0FGR8-2;A0FGR8-4;A0FGR8-5;A0FGR8-6;F2Z...,N-Term(TMT6plex); S3(Phospho); K6(TMT6plex),na,na,na,...,na;na;na;na;na;na;na;na;na;na;na;na,SAQVKRPSVSKEGRK,A0FGR8_S676,18009477,NaN,NaN,NaN,NaN,NaN,NaN
3,sSSsLLASPGHISVk_S(1): <10%; S(2): <10%; S(3): ...,A0FGR8;A0FGR8-2;A0FGR8-5;A0FGR8-6;F2Z3K9;H7BXI1,ESYT2,A0FGR8_S739;A0FGR8-2_S711;A0FGR8-5_S146;A0FGR8...,Extended synaptotagmin-2 OS=Homo sapiens GN=ES...,A0FGR8;A0FGR8-2;A0FGR8-5;A0FGR8-6;F2Z3K9;H7BXI1,N-Term(iTRAQ4plex); S4(Phospho); K15(iTRAQ4plex),1.394,1.370,1.390,...,na;na;na;na;na;na;na;na;na;na;na;na,DLGRSSSSLLASPGH,A0FGR8_S739,468073,NaN,NaN,NaN,NaN,NaN,NaN
4,sSsSLLASPGHISVk_S(1): <10%; S(2): <10%; S(3): ...,A0FGR8;A0FGR8-2;A0FGR8-5;A0FGR8-6;F2Z3K9;H7BXI1,ESYT2,A0FGR8_S738;A0FGR8-2_S710;A0FGR8-5_S145;A0FGR8...,Extended synaptotagmin-2 OS=Homo sapiens GN=ES...,A0FGR8;A0FGR8-2;A0FGR8-5;A0FGR8-6;F2Z3K9;H7BXI1,N-Term(iTRAQ4plex); S3(Phospho); K15(iTRAQ4plex),2.101,2.075,1.601,...,na;na;na;na;na;na;na;na;na;na;na;na,HDLGRSSSSLLASPG,"A0FGR8_S738, A0FGR8-2_S710",468072,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11899,gIGtPQPDVAk_T(4): >90%,Q9Y6X8;H0YKA3,ZHX2,Q9Y6X8_T37;H0YKA3_T37,Zinc fingers and homeoboxes protein 2 OS=Homo ...,Q9Y6X8;H0YKA3,N-Term(TMT6plex); T4(Phospho); K11(TMT6plex),na,na,na,...,na;na;na;na;na;na;na;na;na;na;na;na,AKEKGIGTPQPDVAK,Q9Y6X8_T37,27458804,NaN,NaN,NaN,NaN,NaN,NaN
11900,sVAVsDEEEVEEEAER_S(1): <10%; S(5): >90%,Q9Y6X9;Q9Y6X9-2,MORC2,Q9Y6X9_S743;Q9Y6X9-2_S681,MORC family CW-type zinc finger protein 2 OS=H...,Q9Y6X9;Q9Y6X9-2,N-Term(TMT6plex); S5(Phospho),na,na,na,...,na;na;na;na;na;na;na;na;na;na;na;na,RKRSVAVSDEEEVEE,Q9Y6X9_S743,449185,NaN,NaN,NaN,NaN,NaN,NaN
11901,nSPQSsPTSTPk_S(2): <10%; S(5): <10%; S(6): >90...,Q9Y6Y0;A8MVR0,IVNS1ABP,Q9Y6Y0_S326;A8MVR0_S108,Influenza virus NS1A-binding protein OS=Homo s...,Q9Y6Y0;A8MVR0,N-Term(TMT6plex); S6(Phospho); K12(TMT6plex),na,na,na,...,na;na;na;na;na;na;na;na;na;na;na;na,GRNSPQSSPTSTPKL,Q9Y6Y0_S326,483520,NaN,NaN,NaN,NaN,NaN,NaN
11902,nsPQSSPTSTPk_S(2): >90%; S(5): <10%; S(6): <10...,Q9Y6Y0;A8MVR0,IVNS1ABP,Q9Y6Y0_S322;A8MVR0_S104,Influenza virus NS1A-binding protein OS=Homo s...,Q9Y6Y0;A8MVR0,N-Term(TMT6plex); S2(Phospho); K12(TMT6plex),na,na,na,...,na;na;na;na;na;na;na;na;na;na;na;na,IFLHGRNSPQSSPTS,Q9Y6Y0_S322,483519,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# Extract only phosphosites with >90% localization
def extract_confident_sites(row):
    if isinstance(row, str):
        return re.findall(r'([STY])\((\d+)\):\s*>\s*90%', row)
    return []

# Apply to column
data['Confident_Sites'] = data['Sequence_Localization Probability'].apply(extract_confident_sites)

# Remove rows with no confident sites
data = data[data['Confident_Sites'].str.len() == 1].copy()

# Extract amino acid + position from tuple
data['Amino_Acid'] = data['Confident_Sites'].apply(lambda x: x[0][0])
data['Position'] = data['Confident_Sites'].apply(lambda x: int(x[0][1]))

data.drop(columns=['Confident_Sites'], inplace=True)

In [33]:
data[['GeneNames', 'Amino_Acid', 'Position']].head(10)


,GeneNames,Amino_Acid,Position
0,E2F8,S,3
2,ESYT2,S,3
3,ESYT2,S,4
4,ESYT2,S,3
7,UHRF1BP1L,S,4
8,ARHGEF37,S,8
9,ARHGEF37,S,3
10,PLEKHG3,S,6
11,FSD2,S,3
13,CXorf23,S,3


*SPLIT SEQUENCE LOCALIZATION PROBABILITY TO MULTIPLE COLUMNS(4) -IGNORE ROWS WITH TWO >90% 

In [34]:
# filter data
data['Sequence window'] = data['Sequence Window'].str.replace('_', '')

In [35]:
def match_seq_to_genename(dataset, seq_column):
    '''
    Maps amino acid sequences to gene names using the loaded fasta file.
    
    args:
    =====
    dataset: <pd.Dataframe> with a column of amino acid sequences
    seq_column: <str> column name containing amino acid sequences
    
    out:
    ====
    dataset: <pd.Dataframe> with an additional column containing gene names
    '''    
    
    fasta_sequence = list(SeqIO.parse(f"/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/UP000005640_9606.fasta", "fasta"))
    
    gene_dict = {}
    
    # iterate over rows in seq_column
    for i in dataset[seq_column]:
        print(i)
        i_str = str(i)
        for seq_record in fasta_sequence:
            matches = re.findall(i_str, str(seq_record.seq))
            if matches:
                print(f"Match found for sequence: {seq_record}")
                gene_name_match = re.search(r"GN=(\w+)", seq_record.description)
                print('Gene name match:', gene_name_match)
                if gene_name_match:
                    gene_name = gene_name_match.group(1)
                    gene_dict[i] = gene_name
                    print(f"Match found: {i_str} -> {gene_name}")
                else:
                    print(f"No gene name match found in description for sequence: {i_str}")

    # map sequences to gene names           
    dataset['GeneName'] = dataset[seq_column].map(gene_dict) 
    print('Amino acid sequences matched to gene names.')

    return dataset

In [36]:
data = match_seq_to_genename(data, 'Sequence Window')

ERGSKRASMLEDSGS
Match found for sequence: ID: sp|A0AVK6|E2F8_HUMAN
Name: sp|A0AVK6|E2F8_HUMAN
Description: sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8 OS=Homo sapiens OX=9606 GN=E2F8 PE=1 SV=1
Number of features: 0
Seq('MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFGPLTTPTKPKEGSQG...DVH')
Gene name match: <re.Match object; span=(71, 78), match='GN=E2F8'>
Match found: ERGSKRASMLEDSGS -> E2F8
SAQVKRPSVSKEGRK
Match found for sequence: ID: sp|A0FGR8|ESYT2_HUMAN
Name: sp|A0FGR8|ESYT2_HUMAN
Description: sp|A0FGR8|ESYT2_HUMAN Extended synaptotagmin-2 OS=Homo sapiens OX=9606 GN=ESYT2 PE=1 SV=1
Number of features: 0
Seq('MTANRDAALSSHRHPGCAQRPRTPTFASSSQRRSAFGFDDGNFPGLGERSHAPG...AMT')
Gene name match: <re.Match object; span=(71, 79), match='GN=ESYT2'>
Match found: SAQVKRPSVSKEGRK -> ESYT2
DLGRSSSSLLASPGH
Match found for sequence: ID: sp|A0FGR8|ESYT2_HUMAN
Name: sp|A0FGR8|ESYT2_HUMAN
Description: sp|A0FGR8|ESYT2_HUMAN Extended synaptotagmin-2 OS=Homo sapiens OX=9606 GN=ESYT2 PE=1 SV=1
Number of featur

KeyboardInterrupt: 

In [ ]:
data

,Sequence_Localization Probability,Proteins,GeneNames,Phosphorylation position (>90% localization),Protein Descriptions,Protein Group Accessions,Modifications,iTRAQ Subject 1 115/114 (Exercise/Basal) (normalized),iTRAQ Subject 2 117/116 (Exercise/Basal) (normalized),iTRAQ Subject 3 116/117 (Exercise/Basal) (normalized),...,In Domain,On Function,On Process,On Protein Interaction,On Other Interaction,Confident_Sites,Amino_Acid,Position,Sequence window,GeneName
0,rAsmLEDSGSk_S(3): >90%; S(8): <10%; S(10): <10%,A0AVK6,E2F8,A0AVK6_S608,Transcription factor E2F8 OS=Homo sapiens GN=E...,A0AVK6,N-Term(TMT6plex); S3(Phospho); M4(Oxidation); ...,na,na,na,...,NaN,NaN,NaN,NaN,NaN,"[(S, 3)]",S,3,ERGSKRASMLEDSGS,E2F8
2,rPsVSk_S(3): >90%; S(5): <10%,A0FGR8;A0FGR8-2;A0FGR8-4;A0FGR8-5;A0FGR8-6;F2Z...,ESYT2,A0FGR8_S676;A0FGR8-2_S648;A0FGR8-4_S472;A0FGR8...,Extended synaptotagmin-2 OS=Homo sapiens GN=ES...,A0FGR8;A0FGR8-2;A0FGR8-4;A0FGR8-5;A0FGR8-6;F2Z...,N-Term(TMT6plex); S3(Phospho); K6(TMT6plex),na,na,na,...,NaN,NaN,NaN,NaN,NaN,"[(S, 3)]",S,3,SAQVKRPSVSKEGRK,ESYT2
3,sSSsLLASPGHISVk_S(1): <10%; S(2): <10%; S(3): ...,A0FGR8;A0FGR8-2;A0FGR8-5;A0FGR8-6;F2Z3K9;H7BXI1,ESYT2,A0FGR8_S739;A0FGR8-2_S711;A0FGR8-5_S146;A0FGR8...,Extended synaptotagmin-2 OS=Homo sapiens GN=ES...,A0FGR8;A0FGR8-2;A0FGR8-5;A0FGR8-6;F2Z3K9;H7BXI1,N-Term(iTRAQ4plex); S4(Phospho); K15(iTRAQ4plex),1.394,1.370,1.390,...,NaN,NaN,NaN,NaN,NaN,"[(S, 4)]",S,4,DLGRSSSSLLASPGH,ESYT2
4,sSsSLLASPGHISVk_S(1): <10%; S(2): <10%; S(3): ...,A0FGR8;A0FGR8-2;A0FGR8-5;A0FGR8-6;F2Z3K9;H7BXI1,ESYT2,A0FGR8_S738;A0FGR8-2_S710;A0FGR8-5_S145;A0FGR8...,Extended synaptotagmin-2 OS=Homo sapiens GN=ES...,A0FGR8;A0FGR8-2;A0FGR8-5;A0FGR8-6;F2Z3K9;H7BXI1,N-Term(iTRAQ4plex); S3(Phospho); K15(iTRAQ4plex),2.101,2.075,1.601,...,NaN,NaN,NaN,NaN,NaN,"[(S, 3)]",S,3,HDLGRSSSSLLASPG,ESYT2
7,sQSsLSGkPk_S(1): <10%; S(3): <10%; S(4): >90%;...,A0JNW5,UHRF1BP1L,A0JNW5_S1081,UHRF1-binding protein 1-like OS=Homo sapiens G...,A0JNW5,N-Term(TMT6plex); S4(Phospho); K8(TMT6plex); K...,na,na,na,...,NaN,NaN,NaN,NaN,NaN,"[(S, 4)]",S,4,RTLKSQSSLSGKPKE,BLTP3B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11898,gsGLAGPk_S(2): >90%,Q9Y6W5,WASF2,Q9Y6W5_S284,Wiskott-Aldrich syndrome protein family member...,Q9Y6W5,N-Term(TMT6plex); S2(Phospho); K8(TMT6plex),na,na,na,...,NaN,NaN,NaN,NaN,NaN,"[(S, 2)]",S,2,PVDNQRGSGLAGPKR,WASF2
11899,gIGtPQPDVAk_T(4): >90%,Q9Y6X8;H0YKA3,ZHX2,Q9Y6X8_T37;H0YKA3_T37,Zinc fingers and homeoboxes protein 2 OS=Homo ...,Q9Y6X8;H0YKA3,N-Term(TMT6plex); T4(Phospho); K11(TMT6plex),na,na,na,...,NaN,NaN,NaN,NaN,NaN,"[(T, 4)]",T,4,AKEKGIGTPQPDVAK,ZHX2
11900,sVAVsDEEEVEEEAER_S(1): <10%; S(5): >90%,Q9Y6X9;Q9Y6X9-2,MORC2,Q9Y6X9_S743;Q9Y6X9-2_S681,MORC family CW-type zinc finger protein 2 OS=H...,Q9Y6X9;Q9Y6X9-2,N-Term(TMT6plex); S5(Phospho),na,na,na,...,NaN,NaN,NaN,NaN,NaN,"[(S, 5)]",S,5,RKRSVAVSDEEEVEE,MORC2
11901,nSPQSsPTSTPk_S(2): <10%; S(5): <10%; S(6): >90...,Q9Y6Y0;A8MVR0,IVNS1ABP,Q9Y6Y0_S326;A8MVR0_S108,Influenza virus NS1A-binding protein OS=Homo s...,Q9Y6Y0;A8MVR0,N-Term(TMT6plex); S6(Phospho); K12(TMT6plex),na,na,na,...,NaN,NaN,NaN,NaN,NaN,"[(S, 6)]",S,6,GRNSPQSSPTSTPKL,IVNS1ABP


In [8]:
# Ensure GeneName exists before proceeding
if 'GeneName' not in data.columns:
    raise ValueError("GeneName column is missing! Check match_seq_to_genename function.")


In [ ]:
data['Phosphosite'] = data['Amino_Acid'] + '(' + data['Position'].astype(str) + ')'

# Debugging: Check if 'Phosphosite' column is present after creation
if 'Phosphosite' not in data.columns:
    print("Error: 'Phosphosite' column not created!")
else:
    print("Phosphosite column created successfully.")

# Keep relevant columns
final_data = data[['GeneName', 'Amino_Acid', 'Position', 'Phosphosite']]
final_data

Phosphosite column created successfully.


,GeneName,Amino_Acid,Position,Phosphosite
0,E2F8,S,3,S(3)
2,ESYT2,S,3,S(3)
3,ESYT2,S,4,S(4)
4,ESYT2,S,3,S(3)
7,BLTP3B,S,4,S(4)
...,...,...,...,...
11898,WASF2,S,2,S(2)
11899,ZHX2,T,4,T(4)
11900,MORC2,S,5,S(5)
11901,IVNS1ABP,S,6,S(6)


In [ ]:
# Keep only 'Phosphosite' and columns
keepcols = ['Phosphosite', 'GeneName'] + [col for col in data.columns if col.startswith('iTRAQ') or col.startswith('TMT')]
data = data[keepcols]
data

,Phosphosite,GeneName,iTRAQ Subject 1 115/114 (Exercise/Basal) (normalized),iTRAQ Subject 2 117/116 (Exercise/Basal) (normalized),iTRAQ Subject 3 116/117 (Exercise/Basal) (normalized),iTRAQ Subject 4 114/115 (Exercise/Basal) (normalized),iTRAQ Log Fold-Change,iTRAQ Adjusted P-Value,TMT Subject 1 127_N/126 (Exercise/Basal) (normalized),TMT Subject 2 128_N/127_C (Exercise/Basal) (normalized),TMT Subject 3 129_N/128_C (Exercise/Basal) (normalized),TMT Subject 4 130_N/129_C (Exercise/Basal) (normalized),TMT Pool 131/130_C (Exercise/Basal) (normalized),TMT Log Fold-Change,TMT Adjusted P-Value,iTRAQ and TMT Combined Fold-Change,iTRAQ and TMT Fisher's Combined P-Value
0,S(3),E2F8,na,na,na,na,na,na,2.000,1.495,1.425,1.285,1.459,0.684,0.056,0.684,na
2,S(3),ESYT2,na,na,na,na,na,na,1.880,1.832,2.320,1.707,1.932,0.961,0.007,0.961,na
3,S(4),ESYT2,1.394,1.370,1.390,1.155,0.428,0.045,1.609,1.641,2.068,1.330,0.917,0.727,0.013,0.578,0.001
4,S(3),ESYT2,2.101,2.075,1.601,1.448,0.869,0.016,na,na,na,na,na,na,na,0.869,na
7,S(4),BLTP3B,na,na,na,na,na,na,1.180,1.308,1.243,0.880,1.169,0.198,0.230,0.198,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11898,S(2),WASF2,na,na,na,na,na,na,1.235,1.267,1.156,1.219,1.148,0.295,0.035,0.295,na
11899,T(4),ZHX2,na,na,na,na,na,na,1.027,0.908,1.105,0.894,0.856,-0.021,0.867,-0.021,na
11900,S(5),MORC2,na,na,na,na,na,na,0.813,0.615,0.802,0.780,1.021,-0.412,0.019,-0.412,na
11901,S(6),IVNS1ABP,na,na,na,na,na,na,0.937,0.607,0.684,0.455,0.712,-0.567,0.041,-0.567,na


In [ ]:
# Apply log2 transformation to iTRAQ and TMT quantification columns
quant_columns = [col for col in data.columns if col.startswith('iTRAQ') or col.startswith('TMT')]
# Perform log2 transformation
data[quant_columns] = data[quant_columns].apply(pd.to_numeric, errors='coerce')

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_1046/3574221054.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[quant_columns] = data[quant_columns].apply(pd.to_numeric, errors='coerce')


In [12]:
def log2_transform(dataset):
    '''
    Log2 transform a dataset.
    
    args:
    =====
    dataset: <pd.DataFrame>
    
    out:
    ====
    dataset: <pd.DataFrame> with log2 transformed values
    '''
    cols_to_transform = [col for col in data.columns if col.startswith('iTRAQ') or col.startswith('TMT')]
    dataset[cols_to_transform] = np.log2(dataset[cols_to_transform])
    print('Data has been log2 transformed.')
    return dataset

In [ ]:
data = log2_transform(data)
data

Data has been log2 transformed.


/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/.venv/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log2
  result = func(self.values, **kwargs)
/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/.venv/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log2
  result = func(self.values, **kwargs)
/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_1046/4027182878.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[cols_to_transform] = np.log2(dataset[cols_to_transform])


,Phosphosite,GeneName,iTRAQ Subject 1 115/114 (Exercise/Basal) (normalized),iTRAQ Subject 2 117/116 (Exercise/Basal) (normalized),iTRAQ Subject 3 116/117 (Exercise/Basal) (normalized),iTRAQ Subject 4 114/115 (Exercise/Basal) (normalized),iTRAQ Log Fold-Change,iTRAQ Adjusted P-Value,TMT Subject 1 127_N/126 (Exercise/Basal) (normalized),TMT Subject 2 128_N/127_C (Exercise/Basal) (normalized),TMT Subject 3 129_N/128_C (Exercise/Basal) (normalized),TMT Subject 4 130_N/129_C (Exercise/Basal) (normalized),TMT Pool 131/130_C (Exercise/Basal) (normalized),TMT Log Fold-Change,TMT Adjusted P-Value,iTRAQ and TMT Combined Fold-Change,iTRAQ and TMT Fisher's Combined P-Value
0,S(3),E2F8,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.580145,0.510962,0.361768,0.544980,-0.547932,-4.158429,-0.547932,NaN
2,S(3),ESYT2,NaN,NaN,NaN,NaN,NaN,NaN,0.910733,0.873420,1.214125,0.771463,0.950095,-0.057392,-7.158429,-0.057392,NaN
3,S(4),ESYT2,0.479231,0.454176,0.475085,0.207893,-1.224317,-4.473931,0.686164,0.714575,1.048236,0.411426,-0.125006,-0.459973,-6.265345,-0.790859,-9.965784
4,S(3),ESYT2,1.071076,1.053111,0.678973,0.534062,-0.202572,-5.965784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.202572,NaN
7,S(4),BLTP3B,NaN,NaN,NaN,NaN,NaN,NaN,0.238787,0.387363,0.313826,-0.184425,0.225275,-2.336428,-2.120294,-2.336428,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11898,S(2),WASF2,NaN,NaN,NaN,NaN,NaN,NaN,0.304511,0.341417,0.209141,0.285698,0.199123,-1.761213,-4.836501,-1.761213,NaN
11899,T(4),ZHX2,NaN,NaN,NaN,NaN,NaN,NaN,0.038436,-0.139236,0.144046,-0.161653,-0.224317,NaN,-0.205896,NaN,NaN
11900,S(5),MORC2,NaN,NaN,NaN,NaN,NaN,NaN,-0.298673,-0.701342,-0.318326,-0.358454,0.029983,NaN,-5.717857,NaN,NaN
11901,S(6),IVNS1ABP,NaN,NaN,NaN,NaN,NaN,NaN,-0.093879,-0.720232,-0.547932,-1.136062,-0.490051,NaN,-4.608232,NaN,NaN


In [14]:
def create_phos_ID(dataset):
    '''
    Concatenates GeneName and Phosphosite columns.
    
    args:
    =====
    dataset: <pd.Dataframe> with columns 'GeneName' and 'Phosphosite'
    
    out:
    ====
    dataset: <pd.Dataframe> with 'phosphosite_ID' column and 'GeneName' + 'Phosphosite' columns dropped
    '''
    dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)
    dataset = dataset.drop(columns=['Phosphosite', 'GeneName'])
    print('Phosphosite IDs created.')
    return dataset

data = create_phos_ID(data) # call function to create phosphosite_ID column

print('Phosphosite IDs created.')

Phosphosite IDs created.
Phosphosite IDs created.


/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_1046/3553972964.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)


In [15]:
def clean_phosID_col(data):
    data = data[~data.phosphosite_ID.str.contains('nan', case = False)]
    data = data[~data.phosphosite_ID.str.contains(';', case = False)] # remove rows containing ';' in phosphosite_ID column
    data = data[~data.phosphosite_ID.str.contains('-', case = False)] # remove rows containing '-' in phosphosite_ID column
    
    # check whether there are any phosphosites with multiple measurements
    data_grouped = data.groupby(by = 'phosphosite_ID')
    if len(data) != len(data_grouped):
        data = data_grouped.mean()
        data.reset_index(inplace=True) # reset index
        print('Phosphosites with multiple measurements have been averaged')
    else:
        print('There are no phosphosites with multiple measurements')
        
    print(data)
        
    data = data.replace([np.inf, -np.inf], np.nan)
        
    if data.columns[0] != 'phosphosite_ID':
        phosphosite_ID = data.pop('phosphosite_ID')
        data.insert(0, 'phosphosite_ID', phosphosite_ID)
    return data

In [ ]:
data = clean_phosID_col(data)
print("After cleaning phosphosite_ID column:")
data

Phosphosites with multiple measurements have been averaged
     phosphosite_ID  iTRAQ Subject 1 115/114 (Exercise/Basal) (normalized)  \
0         AAAS_S(2)                                                NaN       
1         AAK1_S(9)                                                NaN       
2         AAK1_T(4)                                                NaN       
3        AAMDC_S(7)                                                NaN       
4        ABCA1_S(3)                                                NaN       
...             ...                                                ...       
5997    ZSWIM8_S(4)                                                NaN       
5998       ZYX_S(1)                                                NaN       
5999      ZYX_S(14)                                                NaN       
6000       ZYX_S(6)                                                NaN       
6001       ZYX_S(9)                                                NaN       

    

,phosphosite_ID,iTRAQ Subject 1 115/114 (Exercise/Basal) (normalized),iTRAQ Subject 2 117/116 (Exercise/Basal) (normalized),iTRAQ Subject 3 116/117 (Exercise/Basal) (normalized),iTRAQ Subject 4 114/115 (Exercise/Basal) (normalized),iTRAQ Log Fold-Change,iTRAQ Adjusted P-Value,TMT Subject 1 127_N/126 (Exercise/Basal) (normalized),TMT Subject 2 128_N/127_C (Exercise/Basal) (normalized),TMT Subject 3 129_N/128_C (Exercise/Basal) (normalized),TMT Subject 4 130_N/129_C (Exercise/Basal) (normalized),TMT Pool 131/130_C (Exercise/Basal) (normalized),TMT Log Fold-Change,TMT Adjusted P-Value,iTRAQ and TMT Combined Fold-Change,iTRAQ and TMT Fisher's Combined P-Value
0,AAAS_S(2),NaN,NaN,NaN,NaN,NaN,NaN,-0.174621,-0.411195,-0.329160,-0.345564,-0.109359,NaN,-5.506353,NaN,NaN
1,AAK1_S(9),NaN,NaN,NaN,NaN,NaN,NaN,0.719841,0.341417,0.797428,0.393965,0.348232,-0.736966,-5.011588,-0.736966,NaN
2,AAK1_T(4),NaN,NaN,NaN,NaN,NaN,NaN,-0.087733,-0.029146,0.253384,-0.334607,-0.316528,NaN,-0.498179,NaN,NaN
3,AAMDC_S(7),NaN,NaN,NaN,NaN,NaN,NaN,-0.072483,-0.092340,-0.106249,-0.005782,-0.008682,NaN,-1.158429,NaN,NaN
4,ABCA1_S(3),NaN,NaN,NaN,NaN,NaN,NaN,-0.015958,-0.560643,0.189034,-0.392137,0.017209,NaN,-1.518701,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5997,ZSWIM8_S(4),NaN,NaN,NaN,NaN,NaN,NaN,-0.424688,-0.531156,0.073135,-0.588574,-0.573467,NaN,-3.698998,NaN,NaN
5998,ZYX_S(1),NaN,NaN,NaN,NaN,NaN,NaN,0.511974,0.212881,0.673556,0.116365,0.048236,-1.296899,-3.857260,-1.296899,NaN
5999,ZYX_S(14),NaN,NaN,NaN,NaN,NaN,NaN,-0.220950,-0.492079,-0.057392,-0.590745,-0.291592,NaN,-4.158429,NaN,NaN
6000,ZYX_S(6),NaN,NaN,NaN,NaN,NaN,NaN,-0.924125,-1.101598,-0.573467,-1.289827,-1.055891,NaN,-6.795859,NaN,NaN


In [ ]:
data.to_csv(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/PreprocessedDatasets/NJH2015.csv', index=False)


print(dataset, 'has been saved to CSV successfully!', data)

NJH2015 has been saved to CSV successfully!      phosphosite_ID  iTRAQ Subject 1 115/114 (Exercise/Basal) (normalized)  \
0         AAAS_S(2)                                                NaN       
1         AAK1_S(9)                                                NaN       
2         AAK1_T(4)                                                NaN       
3        AAMDC_S(7)                                                NaN       
4        ABCA1_S(3)                                                NaN       
...             ...                                                ...       
5997    ZSWIM8_S(4)                                                NaN       
5998       ZYX_S(1)                                                NaN       
5999      ZYX_S(14)                                                NaN       
6000       ZYX_S(6)                                                NaN       
6001       ZYX_S(9)                                                NaN       

      iTRAQ Subject